Imports the necessary libraries and sets up data table

In [ ]:
import pandas
import numpy
from sklearn import preprocessing
from sklearn import model_selection
from sklearn import neural_network
from sklearn import metrics
from sklearn import neighbors
from sklearn.preprocessing import StandardScaler
from matplotlib import pyplot as plt


#data = pandas.read_csv('minmax_normalised_trackdata.csv')
#data = pandas.read_csv('dupremoved_trackdata.csv')
#data = pandas.read_csv('softmax_normalised_trackdata.csv')
#data = pandas.read_csv('zscore_normalised_trackdata.csv')
#data = pandas.read_csv('collinearity_removed.csv')
#data = pandas.read_csv('dbscanned_trackdata.csv')

data.info()
genreMap = sorted(data["genre"].unique())

Training and testing set creation 

In [ ]:
#uncomment the first X if using anything that isn't collinearity_removed.csv
#X = data.drop(["name", "artist", "album", "duration_ms", "id", "genre"], axis=1)
#X = data.drop("genre", axis=1)
y = preprocessing.LabelEncoder().fit_transform(data["genre"])


Metrics using neural network

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train = scaler.transform(X_train)  
X_test = scaler.transform(X_test)
model = neural_network.MLPClassifier(activation= 'tanh', max_iter= 10000000).fit(X_train, y_train)  


yPredicted = model.predict(X_test)
metrics.plot_confusion_matrix(model, X_test, y_test, cmap='GnBu')
plt.show()

print('Accuracy: %.3f' % metrics.accuracy_score(y_test, yPredicted))
print('Precision: %.3f' % metrics.precision_score(y_test, yPredicted, average="macro"))
print('Recall: %.3f' % metrics.recall_score(y_test, yPredicted, average="macro")) 
print(genreMap)

Metrics using K neighbors, k clusters found using elbow method

In [ ]:
maxAccuracy = 0
for i in range(1000) :
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.25)
    scaler = StandardScaler()  
    scaler.fit(X_train)  
    X_train = scaler.transform(X_train)  
    X_test = scaler.transform(X_test)
    model = neighbors.KNeighborsClassifier(n_neighbors= i + 1, weights= 'distance', algorithm= 'brute')
    model.fit(X_train, y_train) 
    yPredicted = model.predict(X_test)
    if  metrics.accuracy_score(y_test, yPredicted) > maxAccuracy :
        bestX = X_test
        bestY = y_test
        bestPrediction = yPredicted
        maxAccuracy =  metrics.accuracy_score(y_test, yPredicted)

metrics.plot_confusion_matrix(model, bestX, bestY, cmap='GnBu')
plt.show()

print('Accuracy: %.3f' % maxAccuracy)
print('Precision: %.3f' % metrics.precision_score(bestY, bestPrediction, average="macro"))
print('Recall: %.3f' % metrics.recall_score(bestY, bestPrediction, average="macro")) 
print(genreMap)
